# Interacting with Recommenders, Campaigns and Filters <a class="anchor" id="top"></a>

In this notebook, you will interact with campaigns and filters in Amazon Personalize.

1. [Introduction](#intro)
1. [Interact with Recommenders](#interact-recommenders)
1. [Interact with Campaigns](#interact-campaigns)
1. [Filters](#filters)
1. [Create Filters](#create-filters)
1. [Using Filters](#using-filters)
1. [Real-time Events](#real-time)
1. [Batch Recommendations](#batch)
1. [Wrap Up](#wrapup)

To run this notebook, you need to have run the previous notebooks, [`01_Data_Layer.ipynb`](01_Data_Layer.ipynb), and [`02_Training_Layer_Recap.ipynb`](02_Training_Layer_Recap.ipynb), where you created a dataset and imported interaction, item, and user metadata data into Amazon Personalize, created recommenders, solutions and campaigns.

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have 2 Recommenders and one deployed campaign. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 


In this Notebook we will interact with Recommenders and Campaigns and get recommendations. 

We will create and interact with filters and send live data to Amazon Personalize to see the effect of real-time interactions on recommendations.

![Workflow](images/image3.png)

As you work with Amazon Personalize, you can modify the helper functions to fit the structure of your data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import pandas as pd

In [ ]:
%store -r

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

First, let's create a supporting function to help make sense of the results returned by a Personalize recommender or campaign. Personalize returns only an `item_id`. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the Movielens dataset.

Start by loading in the dataset which we can use for our lookup table.

In [ ]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(data_dir + '/imdb/items.csv', index_col=0)

# Render some sample data
items_df.head(5)

By defining the ID column as the index column it is trivial to return a movie by just querying the ID. Movie #589 should be Terminator 2: Judgment Day.

In [ ]:
movieIdExample = 'tt0093779'
title = items_df.loc[movieIdExample]['TITLE']
print(title)

That isn't terrible, but it would get messy to repeat this everywhere in our code, so the function below will clean that up.

In [ ]:
def get_movie_by_id(movieId, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[movieId]['TITLE']
    except:
        return "Error obtaining title"

Now let's test a few simple values to check our error catching.

In [ ]:
# A known good id (The Princess Bride)
print(get_movie_by_id(movieId="tt0093779"))
# A bad type of value
print(get_movie_by_id(movieId="987.9393939"))
# Really bad values
print(get_movie_by_id(movieId="Steve"))

Great! Now we have a way of rendering results. 

## Interact with recommenders <a class="anchor" id="interact-recommenders"></a>
[Back to top](#top)

Now that the recommenders have been trained, lets have a look at the recommendations we can get for our users!

### "More like X" Recommender

'More like X' requires an item and a user as input, and it will return items which users interact with in similar ways to their interaction with the input item. In this particular case the item is a movie. 

The cells below will handle getting recommendations from the "More like X" Recommender and rendering the results. Let's see what the recommendations are for the first item we looked at earlier in this notebook (Terminator 2: Judgment Day).

We will be using the `recommenderArn`, the `itemId`, the `userId` as well as the number or results we want, `numResults`.

In [ ]:
# First pick a user
testUserId = "1"

In [ ]:
workshop_recommender_more_like_x_arn


In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_more_like_x_arn,
    itemId = 'tt0093779',
    userId = testUserId,
    numResults = 20
)

First lets get the response directly from the get_recommendations API, which by default returns 25 items, but can be adjusted.

In [ ]:
print(get_recommendations_response)

Depending on your Item catalog, this response will not be easy to quickly evaluate the recommendations, so lets use the function we created above.

In [ ]:
itemList = get_recommendations_response['itemList']
print("If you liked " + get_movie_by_id('tt0093779') + ", you may also like:")
print()
for item in itemList:
    print(get_movie_by_id(movieId=item['itemId']))

Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for similar movies render in a nice dataframe. Again, let's create a helper function to achieve this.

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_id, user_id):
    # Get the movie name
    movie_name = get_movie_by_id(movie_id)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_more_like_x_arn,
        itemId = str(movie_id),
        userId = user_id
    )
    # Build a new dataframe of recommendations
    itemList = get_recommendations_response['itemList']
    recommendation_list = []
    for item in itemList:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

Now, let's test the helper function with several different movies. Let's sample some data from our dataset to test our "More like X" Recommender. Grab 5 random movies from our dataframe.

Note: We are going to show similar titles, so you may want to re-run the sample until you recognize some of the movies listed

In [ ]:
samples = items_df.sample(5)
samples

In [ ]:
more_like_x_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    more_like_x_recommendations_df = get_new_recommendations_df(more_like_x_recommendations_df, movie, testUserId)

more_like_x_recommendations_df

You may notice that some of the items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions, which will be more common with the movielens small dataset). 

### "Top picks for you" Recommender

"Top picks for you" supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since "Top picks for you" relies on having a sampling of users, let's load the data we need for that and select 3 random users. Since Movielens does not include user data, we will select 3 random numbers from the range of user id's in the dataset.

In [ ]:
users = random.sample(range(1, 600), 3)

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

"Top picks for you" requires only a user as input, and it will return items that are relevant for that particular user. In this particular case the item is a movie.

The cells below will handle getting recommendations from the "Top picks for you" Recommender and rendering the results. 

We will be using the `recommenderArn`, the `userId` as well as the number or results we want, `numResults`.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_top_picks_arn,
        userId = str(user_id),
        numResults = 20
    )
    # Build a new dataframe of recommendations
    itemList = get_recommendations_response['itemList']
    recommendation_list = []
    for item in itemList:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

In [ ]:
recommendations_df_users = pd.DataFrame()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

## Interact with Campaigns <a class="anchor" id="interact-campaigns"></a>
[Back to top](#top)

Now that the reranking campaign is deployed and active, we can start to get recommendations via an API call. 

### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For a VOD application you want dynamically render a personalized shelf/rail/carousel based on some information (director, location, superhero franchise, movie time period, etc...). This may not be information that you have in your metadata, so an item metadata filter will not work, however you may have this information within you system to generate the item list. 

To demonstrate this, we will use the same user from before and a random collection of items.

In [ ]:
rerank_user = user
rerank_items = items_df.sample(25).index.tolist()

Now build a nice dataframe that shows the input data.

In [ ]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

Then make the personalized ranking API call.

In [ ]:
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = workshop_rerank_campaign_arn,
        userId = str(rerank_user),
        inputList = rerank_item_list
)

Now add the reranked items as a second column to the original dataframe, for a side-by-side comparison.

In [ ]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user that cannot be easily categorized in your metadata, for instance "Critics picks" which are curated by a person.

# Filters <a class="anchor" id="filters"></a>

## Create Filters <a class="anchor" id="create-filters"></a>
[Back to top](#top)

Personalize can utilize either [static or dynamic filters](https://docs.aws.amazon.com/personalize/latest/dg/filter.html). Static filters are where the filter properties are built into the filter itself, which makes invocation simpler, but gives less flexibility. An example of this would be a "Horror" movie filter, which invokes the get_recommendations_response api with the specific filter of GENRE = Horror. In order to create a recommendation for each filter that would require 10+ filters. Personalize also supports dynamic filters, where the values can be passed at runtime, allowing for a single filter of GENRE, where the actual genre is passed at runtime. 

Filters can be created for fields of both Items and Events. 

A few common use cases for dynamic filters in Video On Demand are:

Categorical filters based on Item Metadata - Often your item metadata will have information about the title such as Genre, Keyword, Year, Director, Actor etc. Filtering on these can provide recommendations within that data, such as action movies, Steven Spielberg movies, Movies from 1995 etc.

Range based filters based on Item Metadata - Personalize supports range operations in both static and dynamic filters. Filtering based on a range can be used to create recommendations such as "Whats on now" in live tv scenarios, best of decade, Movies rated over 8/10 stars etc

Events - you may want to filter out certain events and provide results based on those events, such as moving a title from a "suggestions to watch" recommendation to a "watch again" recommendations.


### Create the Genre filter

In [ ]:
# Create a dataframe for the items by reading in the correct source CSV
items_meta_df = pd.read_csv(data_dir + '/item-meta.csv', sep=',', index_col=0)

# Render some sample data
items_meta_df.head(10)

Since there are a lot of genres to filter on, we will create a dynamic filter using the dynamic variable $GENRE, this will allow us to pass in the variable at runtime rather than create a static filter for each genre.

In [ ]:
filter_name = 'Genre'
try:
    create_genre_filter_response = personalize.create_filter(
        name=filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'INCLUDE ItemID WHERE Items.GENRES IN ($GENRE)'
    )
    
    genre_filter_arn = create_genre_filter_response['filterArn']

    print('Creating the Personalize filter with genre_filter_arn {}.'.format(genre_filter_arn))
    
except personalize.exceptions.ResourceAlreadyExistsException as e:
    genre_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+filter_name
    print('The Personalize filter {} already exists.'.format(filter_name))
    print ('\nWe will be using the existing Personalize Filter with genre_filter_arn = {}'.format(genre_filter_arn))
    

### Create the Promotion filter

In [ ]:
promotion_filter_name = 'promotion'
try:
    create_promotion_filter_response = personalize.create_filter(
        name=promotion_filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'INCLUDE ItemID WHERE Items.PROMOTION IN ("true")'
    )
    
    promotion_filter_arn = create_promotion_filter_response['filterArn']
    print('Creating the Personalize filter with promotion_filter_arn {}.'.format(promotion_filter_arn))
    

except personalize.exceptions.ResourceAlreadyExistsException as e:
    promotion_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+promotion_filter_name
    print('The Personalize filter {} already exists.'.format(promotion_filter_name))
    print ('\nWe will be using the existing Personalize Filter with promotion_filter_arn = {}'.format(promotion_filter_arn))
    

### Create the Year Range filter
Personalize can also filter based on numerical ranges. This can be helpful if you want to look for items that are within a given time window, above a certain rating etc. For that we will create a filter for decades.

In [ ]:
year_range_filter_name = 'YearRange'
try:
    create_genre_filter_response = personalize.create_filter(
        name=year_range_filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'INCLUDE ItemID WHERE Items.YEAR >= $YEAR1 AND Items.YEAR < $YEAR2'
    )
    
    year_range_filter_arn = create_genre_filter_response['filterArn']
    print('Creating the Personalize filter with year_range_filter_arn {}.'.format(year_range_filter_arn))
    

except personalize.exceptions.ResourceAlreadyExistsException as e:
    year_range_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+year_range_filter_name
    print('The Personalize filter {} already exists.'.format(year_range_filter_name))
    print ('\nWe will be using the existing Personalize Filter with year_range_filter_arn = {}'.format(year_range_filter_arn))
    
    
    

### Create filters for Watched and Unwatched movies
Lets also create 2 event filters for watched and unwatched content. The Top picks for you and More like X already have a filter on implemented to filter out watched events.

In [ ]:
watched_filter_name = 'watched'

try:
    create_watched_filter_response = personalize.create_filter(
        name=watched_filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'INCLUDE ItemID WHERE Interactions.event_type IN ("Watch")'
    )
    
    watched_filter_arn = create_watched_filter_response['filterArn']
    print('Creating the Personalize filter with watched_filter_arn {}.'.format(watched_filter_arn))
    

except personalize.exceptions.ResourceAlreadyExistsException as e:
    watched_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+watched_filter_name
    print('The Personalize filter {} already exists.'.format(watched_filter_name))
    print ('\nWe will be using the existing Personalize Filter with watched_filter_arn = {}'.format(watched_filter_arn))
    

unwatched_filter_name = 'unwatched'
try:
    create_unwatched_filter_response = personalize.create_filter(
        name = unwatched_filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'EXCLUDE ItemID WHERE Interactions.event_type IN ("Watch")'
    )
    unwatched_filter_arn = create_unwatched_filter_response['filterArn']
    print('Creating the Personalize filter with unwatched_filter_arn {}.'.format(unwatched_filter_arn))

except personalize.exceptions.ResourceAlreadyExistsException as e:
    unwatched_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+unwatched_filter_name
    print('\nThe Personalize filter {} already exists.'.format(unwatched_filter_name))
    print ('\nWe will be using the existing Personalize Filter with unwatched_filter_arn = {}'.format(unwatched_filter_arn))
    

Before we move on we want to add those filters to a list as well so they can be used later.

In [ ]:
interaction_filter_arns = [watched_filter_arn, unwatched_filter_arn]

In [ ]:
interaction_filter_arns

In [ ]:
max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:

    # Genre filter
    version_response = personalize.describe_filter(
        filterArn = genre_filter_arn
    )
    status_genre_filter = version_response["filter"]["status"]

    if status_genre_filter == "ACTIVE":
        print("Build succeeded for {}".format(genre_filter_arn))
        
    elif status_genre_filter == "CREATE FAILED":
        print("Build failed for {}".format(genre_filter_arn))
        break
        
    if not status_genre_filter == "ACTIVE":
        print("The filter Genre is still in progress")
    else:
        print("The filter Genre is ACTIVE")

    # Promotion filter
    version_response = personalize.describe_filter(
        filterArn = promotion_filter_arn
    )
    status_promotion_filter = version_response["filter"]["status"]

    if status_promotion_filter == "ACTIVE":
        print("Build succeeded for {}".format(promotion_filter_arn))
        
    elif status_promotion_filter == "CREATE FAILED":
        print("Build failed for {}".format(promotion_filter_arn))
        break
        
    if not status_promotion_filter == "ACTIVE":
        print("The filter Promotion is still in progress")
    else:
        print("The filter Promotion is ACTIVE")   
        
        
    # Decade filter
    version_response = personalize.describe_filter(
        filterArn = year_range_filter_arn
    )
    status_year_range_filter = version_response["filter"]["status"]

    if status_year_range_filter == "ACTIVE":
        print("Build succeeded for {}".format(year_range_filter_arn))
        
    elif status_year_range_filter == "CREATE FAILED":
        print("Build failed for {}".format(year_range_filter_arn))
        break
        
    if not status_year_range_filter == "ACTIVE":
        print("The filter YearRange is still in progress")
    else:
        print("The filter YearRange is ACTIVE")

    # Watched filter
    version_response = personalize.describe_filter(
        filterArn = watched_filter_arn
    )
    status_watched_filter = version_response["filter"]["status"]

    if status_watched_filter == "ACTIVE":
        print("Build succeeded for {}".format(watched_filter_arn))
        
    elif status_watched_filter == "CREATE FAILED":
        print("Build failed for {}".format(watched_filter_arn))
        break
        
    if not status_watched_filter == "ACTIVE":
        print("The filter Watched is still in progress")
    else:
        print("The filter Watched is ACTIVE")
 
    # Unatched filter
    version_response = personalize.describe_filter(
        filterArn = unwatched_filter_arn
    )
    status_unwatched_filter = version_response["filter"]["status"]

    if status_unwatched_filter == "ACTIVE":
        print("Build succeeded for {}".format(unwatched_filter_arn))
        
    elif status_unwatched_filter == "CREATE FAILED":
        print("Build failed for {}".format(unwatched_filter_arn))
        break
        
    if not status_watched_filter == "ACTIVE":
        print("The filter Unwatched is still in progress")
    else:
        print("The filter Unwatched is ACTIVE")
    
    if status_genre_filter == "ACTIVE" and status_year_range_filter == 'ACTIVE' and status_promotion_filter  == 'ACTIVE' and status_watched_filter == "ACTIVE" and status_unwatched_filter == "ACTIVE":
        break

    print()
    time.sleep(30)

## Using Filters <a class="anchor" id="using-filters"></a>
[Back to top](#top)

Now that the Filters have been created we can use them to filter our recommendations.

### Get recommendations using a static filter

Earlier we chose to promote a specific set of movies, which we created the item metadata field of "PROMOTION". We will use a static filter to ensure that these items are retured in recommendations. First we will use a static filter to ensure that ALL recommendations meet this criteria of the filter.

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_w_promotions_only_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_top_picks_arn,
        userId = str(user_id),
        filterArn = promotion_filter_arn,
        numResults = 20,
    )
    # Build a new dataframe of recommendations
    itemList = get_recommendations_response['itemList']
    recommendation_list = []
    for item in itemList:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

In [ ]:
recommendations_df_users = pd.DataFrame()

for user in users:
    recommendations_df_users = get_new_recommendations_w_promotions_only_df_users(recommendations_df_users, user)

recommendations_df_users

In this scenario for the 3 test users we got recommendations for previously, the static filter limits the recommendations to only those that meet the filter criteria. This does not provide a good balance of recommendations that are relevant and recommendations that meet the filter criteria.

### Get recommendations using a promotions filter

Personalize has a capability to "promote" certain items, which can be defined as a filter. A configurable subset of the recommendations retured will be from the items defined as promotion, in this case 30%.

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_w_promotions_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_top_picks_arn,
        userId = str(user_id),
        numResults = 20,
        promotions = [{
        "name" : "vegas_promotion",
        "percentPromotedItems" : 30,
        "filterArn": promotion_filter_arn,
        }]
    )
    # Build a new dataframe of recommendations
    itemList = get_recommendations_response['itemList']
    
    recommendation_list = []
    for item in itemList:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

In [ ]:
recommendations_df_users = pd.DataFrame()

for user in users:
    recommendations_df_users = get_new_recommendations_w_promotions_df_users(recommendations_df_users, user)

recommendations_df_users

### Get recommendations using the date range filter

First lets create functions to get recommendations and pass in dynamic filter values.


In [ ]:
def get_new_recommendations_df_by_year_range_filter(recommendations_df, user_id, year_range_filter_arn, filter_value1, filter_value2):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_top_picks_arn,
        userId = str(user_id),
        filterArn = year_range_filter_arn,
        filterValues = {"YEAR1": filter_value1,"YEAR2": filter_value2}
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    filter_name = year_range_filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_value1])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [ ]:
decades_to_filter = [1930,1940,1950,1960,1970,1980,1990,2000,2010]

In [ ]:
# Iterate through Decades
recommendations_df_decade_shelves = pd.DataFrame()
for decade in decades_to_filter:
    recommendations_df_decade_shelves = get_new_recommendations_df_by_year_range_filter(recommendations_df_decade_shelves, user, year_range_filter_arn , str(decade), str(decade+10))
    
recommendations_df_decade_shelves

### Get recommendations unsing the genre filter

In [ ]:
def get_new_recommendations_df_by_genre_filter(recommendations_df, user_id, genre_filter_arn, filter_values):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_top_picks_arn,
        userId = str(user_id),
        filterArn = genre_filter_arn,
        filterValues = { "GENRE": "\"" + filter_values + "\""}
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    filter_name = genre_filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_values])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [ ]:
# Create a dataframe for the items by reading in the correct source CSV
items_meta_df = items_df
# Render some sample data
items_meta_df.head(5)

Now what we want to do is determine the genres to filter on, for that we need a list of all genres. First we will get all the unique values of the column GENRE, then split strings on | if they exist, everyone will then get added to a long list which will be converted to a set for efficiency. That set will then be made into a list so that it can be iterated, and we can then use the get recommendatioins API.

In [ ]:
unique_genre_field_values = items_meta_df['GENRES'].unique()

genre_val_list = []

def process_for_bar_char(val, val_list):
    if '|' in val:
        values = val.split('|')
        for item in values:
            val_list.append(item)
    elif '(' in val:
        pass
    else:
        val_list.append(val)
    return val_list
    

for val in unique_genre_field_values:
    genre_val_list = process_for_bar_char(val, genre_val_list)

genres_to_filter = list(set(genre_val_list))

In [ ]:
genres_to_filter

In [ ]:
# Iterate through Genres
recommendations_df_genre_shelves = pd.DataFrame()
for genre in genres_to_filter:
    recommendations_df_genre_shelves = get_new_recommendations_df_by_genre_filter(recommendations_df_genre_shelves, user, genre_filter_arn , genre)
    
recommendations_df_genre_shelves

## Real-time Events<a class="anchor" id="real-time"></a>
[Back to top](#top)

The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in media workloads, like video-on-demand, where a customer's intent may differ based on if they are watching with their children or on their own.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

Start by creating an event tracker that is attached to the dataset group. This event tracker will add information to the dataset and will influence the recommendations.

In [ ]:
event_tracker_name = 'MovieTracker'
try: 
    create_event_tracker_response = personalize.create_event_tracker(
        name = event_tracker_name,
        datasetGroupArn=workshop_dataset_group_arn
        )
    event_tracker_arn = create_event_tracker_response['eventTrackerArn']
    print(json.dumps(create_event_tracker_response, indent=2))
    print ('\nCreating the event_tracker with event_tracker_arn = {}'.format(event_tracker_arn))
    trackingId = create_event_tracker_response['trackingId']
    print ('\nAnd trackingId = {}'.format(trackingId))
    

except personalize.exceptions.ResourceAlreadyExistsException as e:
    event_tracker_list = personalize.list_event_trackers( 
        datasetGroupArn= workshop_dataset_group_arn
    )['eventTrackers']
    
    event_tracker_arn = event_tracker_list[0]['eventTrackerArn']
    
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn=event_tracker_arn
    )
    trackingId = describe_event_tracker_response['eventTracker']['trackingId']
    
    print ('\nThe the Event Tracker with event_tracker_name {} already exists'.format(event_tracker_name))
    print ('\nWe will be using the existing Event Tracker with event_tracker_arn = {}'.format(event_tracker_arn))
    print ('\nAnd trackingId = {}'.format(trackingId))
    


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:

    version_response = personalize.describe_event_tracker(
        eventTrackerArn = event_tracker_arn
    )
    status = version_response['eventTracker']['status']

    if status == 'ACTIVE':
        print('Build succeeded for {}'.format(event_tracker_arn))
    elif status == "CREATE FAILED":
        print('Build failed for {}'.format(wevent_tracker_arn))
        in_progress_campaigns.remove(event_tracker_arn)
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
    else:
        print('The event tracker build is still in progress')
        
    time.sleep(60)

We will create some code that simulates a user interacting with a particular item. After running this code, you will get recommendations that differ from the results above.

We start by creating some methods for the simulation of real time events.

In [ ]:
sessionDict = {}

def send_movie_click(userId, itemId, eventType):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        sessionId = sessionDict[str(userId)]
    except:
        sessionDict[str(userId)] = str(uuid.uuid1())
        sessionId = sessionDict[str(userId)]
        
    # Configure Properties:
    event = {
    "itemId": str(itemId),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = trackingId,
    userId= str(userId),
    sessionId = sessionId,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(eventType),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, userId, itemId, eventType):
    # Get the artist name (header of column)
    movieName = get_movie_by_id(itemId)
    
    # Interact with different movies
    print('sending event ' + eventType + ' for ' + get_movie_by_id(itemId))
    send_movie_click(userId=userId, itemId=itemId,eventType=eventType)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_top_picks_arn,
        userId = str(userId),
    )
    # Build a new dataframe of recommendations
    itemList = get_recommendations_response['itemList']
    recommendation_list = []
    for item in itemList:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [movieName])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

At this point, we haven't generated any real-time events yet; we have only set up the code. To compare the recommendations before and after the real-time events, let's pick one user and generate the original recommendations for them.

In [ ]:
# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_top_picks_arn,
        userId = str(rerank_user),
    )

# Build a new dataframe for the recommendations
itemList = get_recommendations_response['itemList']
recommendationList = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendationList.append(artist)
user_recommendations_df = pd.DataFrame(recommendationList, columns = [rerank_user])
user_recommendations_df

Now we have a list of recommendations for this user before we have applied any real-time events. Now let's pick 3 random artists which we will simulate our user interacting with, and then see how this changes the recommendations.

In [ ]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [ ]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, rerank_user, movie,'click')
    time.sleep(5)
    

Now we can look at how the click events changed the recommendations.

In [ ]:
user_recommendations_df

In the cell above, the first column after the index is the user's default recommendations from the "Top pics for you" recommender, and each column after that has as a header of the movie that they interacted with via a real time event, and the recommendations after this event occurred. 

The behavior may not shift very much or a lot; this is due to the relatively limited nature of this dataset and effect of a few random clicks. If you wanted to better understand this, try simulating clicking more movies to see the impact.

### Filtering on user's recent interactions
Now lets look at the event filters, which allow you to filter items based on the interaction data. For this dataset, it could be click or watch based on the data we imported, but could be based on whatever interaction schema you design (click, rate, like, watch, purchase etc.) 

We will create a new helper function to use the personalized ranking campaign, sice the Recommenders already filter out watched content.

In [ ]:
def get_new_ranked_recommendations_df_by_static_filter(recommendations_df, user_id, rerank_item_list, filter_arn):
    
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_personalized_ranking(
        campaignArn = workshop_rerank_campaign_arn,
        userId = str(user_id),
        inputList = rerank_item_list,
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['personalizedRanking']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)

    filter_name = filter_arn.split('/')[1]
    new_rec_df = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

In [ ]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_ranked_recommendations_df_by_static_filter(recommendations_df_events, rerank_user, rerank_item_list, filter_arn)
    
recommendations_df_events

Now lets send a watch event in for the top 4 unwatched recommendations, which would simulate watching 4 movies. In a VOD application, you may choose to send in an event after they have watched a significant amount (over 75%) of a piece of content. Sending at 100% complete could miss people that stop short of the credits.

In [ ]:
ranked_unwatched_recommendations_response = personalize_runtime.get_personalized_ranking(
    campaignArn = workshop_rerank_campaign_arn,
    userId = str(rerank_user),
    inputList = rerank_item_list,
    filterArn = filter_arn)

item_list = ranked_unwatched_recommendations_response['personalizedRanking'][:4]

for item in item_list:
    print('sending event Watch for ' + get_movie_by_id(item['itemId']))
    send_movie_click(userId=rerank_user, itemId=item['itemId'], eventType='Watch')
    time.sleep(10)

Now we can look at the event filters to see the updated watched and unwatched recommendations 

In [ ]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_ranked_recommendations_df_by_static_filter(recommendations_df_events, rerank_user, rerank_item_list, filter_arn)
recommendations_df_events

###### Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, make a pull request and provide any additional helpful components that may be missing from this collection.

You can choose to head to [`05_Operations_Layer.ipynb`](05_Operations_Layer.ipynb) to go deeper into ML Ops and what a production solution can look like with an automation pipeline.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in [`06_Clean_Up.ipynb`](06_Clean_Up.ipynb).

In [ ]:
%store workshop_dataset_group_arn
%store region
%store role_name